# CDAT Migration Regression Testing Notebook (`.png` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.


## Setup Code


In [15]:
import glob

from auxiliary_tools.cdat_regression_testing.utils import get_image_diffs

DEV_DIR = "843-migration-phase3"
DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{DEV_DIR}/"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "**/**/*.png"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "v2_9_0_all_sets_model_vs_model"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "**/**/*.png"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [16]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_PATH, DEV_PATH)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_PATH, MAIN_PATH)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

In [17]:
len(DEV_GLOB), len(MAIN_GLOB)

(635, 143)

## 1. Check for matching and equal number of files


In [9]:
_check_if_files_found()

In [10]:
missing_dev_files, missing_main_files = _check_if_missing_files()

In [11]:
missing_dev_files

[]

In [12]:
missing_main_files

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-ALBEDO-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-ALBEDOC-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-ALBEDO_SRF-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-AODDUST-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-AODVIS-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/annual_cycle_zonal_mean/model_vs_model/-ERFtot-ANNUALCYCLE-global.png',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/mai

In [13]:
_check_if_matching_filecount()

OSError: Number of files do not match at DEV_PATH and MAIN_PATH (494 vs. 143).

## 2 Compare the plots between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [14]:
for main_path in MAIN_GLOB:
    dev_path = main_path.replace(MAIN_PATH, DEV_PATH)
    print("Comparing:")
    print(f"    * {main_path}")
    print(f"    * {dev_path}")

    get_image_diffs(dev_path, main_path)

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/area_mean_time_series/FLUT/FLUT.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3/area_mean_time_series/FLUT/FLUT.png
     * Difference path /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3/area_mean_time_series/FLUT_diff/FLUT.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/area_mean_time_series/FSNTOA/FSNTOA.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3/area_mean_time_series/FSNTOA/FSNTOA.png
     * Difference path /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3/area_mean_time_series/FSNTOA_diff/FSNTOA.png
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/v2_9_0_all_sets_model_vs_model/area_mean_time_series/LHFLX/LHFLX.png
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3/area_mean_time_serie

### Results

All plots are identical
